In [1]:

import numpy as np
import collections.abc 
from time import sleep

import googlemaps
import re
import requests
import json
import geopandas
import cartoframes
import folium
import pandas as pd
import getpass
from pymongo import MongoClient
from dotenv import load_dotenv
from bson import json_util
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster

In [2]:
load_dotenv
maps_token = getpass.getpass("maps_token: ")
map_client = googlemaps.Client(maps_token)

API queries_quota: 60


In [3]:
def extracting_data(business,keyword, radius):
    sleep(1)
    location = (41.4036522,2.189654)  
    radius = radius # radius in meters
    business = business
    keyword = keyword
    results = []
    response = map_client.places_nearby(location=location, type=business, radius=radius, keyword=keyword)
    results.extend(response['results'])
    while True:
        try:
            sleep(2)
            response = map_client.places_nearby(location=location, type=business, radius=radius, keyword=keyword, page_token=next_page)
            results.extend(response['results'])
            next_page = response['next_page_token']
        except:
            break
    return results    

In [4]:
def dataframe(data, business, keyword, color, icon_color, icon):
    list_local = []
    df = pd.DataFrame(columns = ['Category','Type', 'Name', 'Address', 'Latitude', 'Longitude', 'Icon', 'Icon Color', 'Color', 'Prefix'])
    for result in data:
        local = {}
        local['Category'] = business
        local['Type'] = keyword
        local['Color'] = color
        local['Icon Color'] = icon_color
        local['Icon'] = icon
        local['Prefix'] = 'fa'
        local['Name'] = result['name']
        local['Address'] = result['vicinity']
        local['Latitude'] = result['geometry']['location']['lat']
        local['Longitude'] = result['geometry']['location']['lng']
        list_local.append(local)
    df = pd.DataFrame(list_local)
    return df    

In [5]:
bars = extracting_data('Bar', 'Bar', 1500)

In [6]:
restaurant = extracting_data('Restaurant', 'Restaurant', 1000)

In [7]:
school = extracting_data('School', 'public School', 2000)

In [8]:
veg_restaurant = extracting_data('Restaurant', 'vegetarian', 500)

In [9]:
hairdresser = extracting_data('hairdresser', 'hairdresser', 300)

In [10]:
tech_startups = extracting_data('tech startups', 'startups', 2000)

In [11]:
coffee = extracting_data('Coffe', 'Coffee Shop', 500)

In [12]:
df_bars = dataframe(bars,'Bar', 'Bar', 'red', 'white', 'beer-mug-empty' )

In [15]:
df_restaurants = dataframe(restaurant,'Restaurant', 'Restaurant', 'orange', 'white', 'utensils')

In [16]:
df_school = dataframe(school, 'School', 'Public School', 'blue', 'black', 'graduation-cap')

In [17]:
df_veg = dataframe(veg_restaurant,'Restaurant', 'Vegetarian', 'green', 'white', 'wheat-awn')

In [18]:
df_hairdresser = dataframe(hairdresser, 'Hairdresser', 'Health & Beauty', 'white', 'black', 'scissors')

In [19]:
df_tech_startups = dataframe(tech_startups, 'Tech startups', 'Startups', 'black', 'white', 'desktop')

In [63]:
df_coffee = dataframe(coffee, 'Coffee', 'Coffee  Shop', 'beige', 'back', 'mug-hot')

In [62]:
basketball = {'Category': 'Sports','Type': 'basketball', 'Name': 'Federació Catalana de Basquetbol', 'Address': 'C. de Guipúscoa, 27,Barcelona', 'Latitude':'41.4144622', 'Longitude':'2.1941339', 'Icon':'basketball', 'Icon Color':'black', 'Color':'lightblue', 'Prefix':'fa'}
index =['0']
df_basketball = pd.DataFrame(basketball, index = index)

In [59]:
company = {'Category': 'Business','Type': 'Tech', 'Name': 'Lessa Soluctions', 'Address': 'Av. Diagonal, 211, Barcelona', 'Latitude':'41.4039759', 'Longitude':'2.1906131', 'Icon':'building', 'Icon Color':'white', 'Color':'black', 'Prefix':'fa'}
df_company = pd.DataFrame(company, index = index)

# Exporting the DataFrame to Mongodb

In [ ]:
def export_to_mongodb(data):
    client = MongoClient("localhost", 27017)
    db = client["Project-Spot"]

    for v in data:
        for c, r in v:
            collection = db[r['Type']]
        result = collection.insert_one(data)

    print(f"Document inserted with ID: {result.inserted_id}")

    return result

### Create the Map layer and add it to the FeatureGroup

# DataFrames
    - df_coffee
    - df_bars 
    - df_restaurants
    - df_school 
    - df_veg
    - df_hairdresser
    - df_tech_startups

In [32]:
main_map = folium.Map(location = ['41.4034532','2.1896495'], zoom_start=15)

In [191]:
def add_spot(data, layer):
    #main_map = folium.Map(location = ['41.4034532','2.1896495'], zoom_start=15)
    
    layer_list = []
    for df, lay in zip(data, layer):
        layer = folium.FeatureGroup(name = f'{lay}')
    folium.Circle(location=['41.4034532', '2.1896495'], radius=1000, color='blue', fill=True).add_to(layer)
    for df in data:
        for k, v in df.iterrows():
            icon = folium.Icon(
                color= v['Color'], 
                icon_color= v['Icon Color'], 
                icon= v['Icon'], prefix= v['Prefix']
                )
            folium.Marker(location=[v['Latitude'], v['Longitude']], popup= v['Name'], icon = icon).add_to(layer)
            #layer.add_child(marker)
            #marker.add_to(layer)
        
        layer_list.append(layer)
        
    #layer.add_to(main_map)
    
    
    return layer_list

In [196]:
main_map = folium.Map(location = ['41.4034532','2.1896495'], zoom_start=15)

In [197]:
city_map = [
    df_coffee, df_tech_startups,
    df_hairdresser, df_veg, 
    df_school, df_bars,
    df_restaurants, df_basketball,
    df_company
]

layers_name = [
    'Bar','Restaurant',
    'Public School','Vegetarian',
    'Health & Beauty','Startups',
    'Coffee  Shop','basketball',
    'Business'
]

In [198]:
layer = add_spot(city_map, layers_name)

In [199]:

for v in layer:
        v.add_to(main_map)
folium.LayerControl().add_to(main_map)
main_map.save('main_map.html')
main_map


In [ ]:
# versao 2.0
def add_spot(data, layer):
    #main_map = folium.Map(location = ['41.4034532','2.1896495'], zoom_start=15)
    
    layer_list = []
    for df, lay in zip(data, layer):
        layer = folium.FeatureGroup(name = f'{lay}')
    folium.Circle(location=['41.4034532', '2.1896495'], radius=1000, color='blue', fill=True).add_to(layer)
    for df in data:
        for k, v in df.iterrows():
            icon = folium.Icon(
                color= v['Color'], 
                icon_color= v['Icon Color'], 
                icon= v['Icon'], prefix= v['Prefix']
                )
            folium.Marker(location=[v['Latitude'], v['Longitude']], popup= v['Name'], icon = icon).add_to(layer)
            #layer.add_child(marker)
            #marker.add_to(layer)
        
        layer_list.append(layer)
        
    #layer.add_to(main_map)
    
    
    return layer_list